In [1]:
import pandas as pd
import numpy as np
from functions import *
from pytorch_util import *
from pytorch_models import ConvBatchLeaky,GRU_NCL,LambdaLayer,biasLayer,Conv2dGLU
from torch.nn import Linear,Sequential,LeakyReLU,Dropout,L1Loss
from torch.utils.data import DataLoader
from torch.optim import Adam

In [2]:
train = pd.read_csv("../Data/train.csv", 
                    dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32})
train = train.values

In [3]:
## change ##
batch_size = 16
clip = .25
block = ResidualBlock3d

Rnn does not work for share weight

Prepare data

In [4]:
train_gen = SequenceGenSpec(train,Is2D=False)
train_gen = DataLoader(train_gen,batch_size,False,num_workers=2)

val_gen = SequenceGenSpec(train,False,Is2D=False)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=2)

Build model

In [80]:
# model = Sequential(   ConvBatchLeaky(1,4,(3,3),stride=(1,1)),
#                       Dropout(0.5),
#                       block(4),
#                       Dropout(0.5),
#                       block(8),
#                       Dropout(0.5),
#                       block(16),
#                       Dropout(0.5),
#                       block(32),
#                       Dropout(0.5),              
#                       LambdaLayer(lambda x:x.view(-1,64*7,47).transpose(1,2)),                  
#                       Linear(64*7,1,bias=False),
#                       biasLayer((1,47,1)),
#                       LeakyReLU(0.2,True),
#                       LambdaLayer(lambda x:x.squeeze(2).median(1)[0])             
#                      ).to('cuda:0')

In [5]:
model = Sequential(   ConvBatchLeaky(1,4,(3,3),stride=(1,1)),
                      Dropout(0.5),
                      block(4),
                      Dropout(0.5),
                      block(8),
                      Dropout(0.5),
                      block(16),
                      Dropout(0.5),
                      block(32),
                      Dropout(0.5),              
                      LambdaLayer(lambda x:x.view(-1,64*7,47).transpose(1,2)),                  
                      Linear(64*7,1,bias=False),
                      LeakyReLU(0.2,True),
                      #LambdaLayer(lambda x:x.squeeze(2).median(1,keepdim=True)[0]),
                      LambdaLayer(lambda x:x.squeeze(2).mean(1,keepdim=True)),                   
                      #biasLayer((1,)),
                      Linear(1,1),
                      LambdaLayer(lambda x:x.squeeze(1))             
                     ).to('cuda:0')

In [33]:
# ResidualBlockGLU3d >> 2.54
# DenseBlock3d >> 2.45
# ResidualBlock3d >> 2.40

In [81]:
opt = Adam(trainable_parameter(model),lr=4e-4)

In [82]:
loss_func_nor = loss_func_generator(L1Loss())

Training

In [8]:
model = fit(10, model, loss_func_nor, opt, train_gen, val_gen,clip=clip)

epoch:0, train_loss:3.478631360599628, val_loss:3.0247867107391357
epoch:1, train_loss:2.2756478207615705, val_loss:2.579779624938965
epoch:2, train_loss:2.152181450564128, val_loss:2.595513343811035
epoch:3, train_loss:2.143061222365269, val_loss:2.532578945159912
epoch:4, train_loss:2.1364198573506794, val_loss:2.4511685371398926
epoch:5, train_loss:2.1179060838543453, val_loss:2.5203874111175537
epoch:6, train_loss:2.1200548811600757, val_loss:2.4278340339660645
epoch:7, train_loss:2.114838432807189, val_loss:2.4521493911743164
epoch:8, train_loss:2.115139267192437, val_loss:2.4467461109161377
epoch:9, train_loss:2.1004444137215614, val_loss:2.4257972240448
Training completed in 69.33237504959106s


Submission

In [10]:
submission = pd.read_csv('../Data/sample_submission.csv')

In [11]:
test_gen = SequenceGenSpecTest(submission.seg_id.tolist(),Is2D=False)
test_gen = DataLoader(test_gen,batch_size,False,num_workers=2)

In [12]:
submission.iloc[:,1] = predict(model,test_gen)

In [13]:
submission.to_csv('../Submission/sub_spec_3d_shareWeight.csv',index=False)